In [1]:
import datetime as dt

import numpy as np
import pandas as pd

from scipy import stats
from io import StringIO

from bokeh.io import output_notebook, show

from lmax_analytics_notebooks.environment.get_creds import get_creds
from lmax_analytics_notebooks.environment.get_env import get_env
from lmax_analytics.dataloader.backtesting_dataloader import BacktestingLoader

from visulisation import Visulisation

output_notebook()

Loading BokehJS ...

In [2]:
creds = get_creds(file="../auth/credentials.yaml")
env = get_env(creds['environment'])

In [3]:
s3 = env.get_boto3_resource(server="blue", key=creds['minio']['key'], secret=creds['minio']['secret'])
fs = env.get_s3fs(server="blue", key=creds['minio']['key'], secret=creds['minio']['secret'])
bl = BacktestingLoader(boto3_fs=s3)
vis = Visulisation()

## ENVIRONMENT VARIABLES
bucket = 'risk-temp'
base_prefix = 'backtesting/simulation_outputs'
shard = 'ldprof'

In [4]:
### SET SCENARIO OR FILENAME DETAILS

In [5]:
# scenario =  {'uid': 'nohedge11', 'version': 1, 'single_file': false, 'start_date': '2021-03-01', 'end_date': '2021-03-05'}
filename = "2021-03-22-simulation-output-ld-bch-testings-2.csv"

In [6]:
### LOAD DATA

In [7]:
if filename:
    print(f'loading single file {filename}')
    buc = s3.Bucket(bucket)
    body = buc.Object(key=f"{base_prefix}/datasource_label={shard}/{filename}").get()["Body"].read()
    df = pd.read_csv(StringIO(str(body, "utf-8")))
    df['timestamp'] = pd.to_datetime(df['timestamp'])
else:
    df = bl.load_sims_s3(
        bucket,
        base_prefix,
        shard, 
        scenario['uid'],
        str(scenario['version']), 
        dt.datetime.strptime(scenario['start_date'], "%Y-%m-%d").date(), 
        dt.datetime.strptime(scenario['end_date'], "%Y-%m-%d").date(), 
        instruments=instruments)

df = df.set_index('timestamp')

loading single file 2021-03-22-simulation-output-ld-bch-testings-2.csv


In [8]:
### LOAD BACKTESTS FOR DATE RANGE AND INSTRUMENT

In [9]:
df.groupby(['hash', 'symbol', 'strategy_position_trigger']).agg({'rpnl_cum': 'last', 'notional_traded': 'sum'}).sort_values('rpnl_cum', ascending=False)

,,,rpnl_cum,notional_traded
hash,symbol,strategy_position_trigger,,
209236733a4721897f590bde12f3cbeb,EUR/USD,1000,23340.00000,3.119657e+09
b362c2c4c941c1af67ca400b061a6060,EUR/USD,500,21300.00000,3.206611e+09
eee9a03fa1f751530e204e95f0a578d0,GBP/USD,750,16005.00000,6.814842e+08
039fdd994c29c57dc70cfa7fcac15197,EUR/USD,750,2985.00000,3.635570e+09
398b1fd7bc964ab5edd95bba0b13c316,USD/CHF,500,2101.66988,1.194808e+07
21fa31a75cd083c5972baeefd6d8dd6e,USD/JPY,1000,-2838.14960,1.201285e+07
0c3339b3dece5f4970c23e38b9278241,USD/JPY,500,-4131.14300,2.499894e+08
c52ba73ef96f1ccf7d375ec1f548996c,GBP/USD,1000,-7620.00000,4.024688e+08
3172c069fab39ff7bd926b768478dbef,GBP/USD,500,-12820.00000,7.777389e+08


In [10]:
### VIEW RESULTS OF BACKTESTING

In [11]:
symbol = 'GBP/USD'

In [12]:
df_subset = df[df.symbol==symbol]

In [13]:
show(vis.plot_sim_profile(df_subset, 'rpnl_cum', list(df_subset.hash.unique()), resample_rule=None))

In [15]:
# df_subset.groupby('hash').first()[['exit_name', 'exit_stoploss_limit', 'exit_takeprofit_limit',
#        'order_book_id', 'portfolio', 'risk_name', 'simulation', 'strategy_max_pos_qty',
#        'strategy_max_pos_qty_buffer', 'strategy_name',
#        'strategy_position_lifespan', 'symbol']]